In [4]:
import dill
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR

def csv_it(dataframe, filebase):
    dataframe.to_csv('./csv/'+filebase+'.csv', sep=',', index=False, encoding='utf-8')
    return

def pkl_it(dataframe, filebase):
    with open('./pkl/'+filebase+'.pkl', 'wb') as fh:
        dill.dump(dataframe, fh)

    return

def sql_it(dataframe, filebase, dbname='zika'): #check!
    postgres_str = 'postgresql://abhijit:{}@127.0.0.1:6742/{}'.format(POSTGRES_PWD, dbname)
    engine = create_engine(postgres_str)
    dataframe.to_sql(filebase, engine, if_exists='replace', dtype={col_name: NVARCHAR for col_name in filebase})
    return

def save_it(dataframe, filebase, dbname='zika'):
    csv_it(dataframe, filebase)
    pkl_it(dataframe, filebase)
    sql_it(dataframe, 'sql_' + filebase, dbname=dbname)
    return

In [5]:
import numpy as np
import os
import pandas as pd
import dill
from datetime import timedelta
from dateutil.parser import parser
import sys
import os
f = 'csv_pkl_sql.py'
for path, dirs, files in os.walk(os.path.expanduser('~/')):    
    if f in files:
        dir_ = path
        break
os.chdir(dir_)
try:
    from csv_pkl_sql import save_it, csv_it, pkl_it
except: 
    print("Change to the project directory")
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


In [6]:
import pandas as pd
import numpy as np
from csv_pkl_sql import csv_it, sql_it
#import cPickle as pickle

from osgeo import gdal


ModuleNotFoundError: No module named 'osgeo'

In [7]:
# Import the latitude and longitude data
lat_long_data = pd.read_csv('./csv/01_latitude_longitude_google.csv')


# Import the geoTIF map
ds = gdal.Open('./gpw_population_density/gpw-v4-population-density_2020.tif')

NameError: name 'gdal' is not defined

In [5]:
ds

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x00000236C5B803F0> >

In [9]:
import pickle
rows = ds.RasterYSize
cols = ds.RasterXSize

transform = ds.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = transform[5]


band = ds.GetRasterBand(1)


def get_population_density(latitude, longitude, 
                           xOrigin=xOrigin, yOrigin=yOrigin,
                           pixelWidth=pixelWidth, pixelHeight=pixelHeight,
                           band=band):

    # # Single point, x=longitude, y=latitude

    x = longitude
    y = latitude

    # This reads three pixels in x- and y- direction
    try:
        #  read 3 x 3 region
        size = 100

        dist_matrix = np.meshgrid(np.arange(-size, size+1), 
                                  np.arange(-size, size+1))
        dist_matrix = np.sqrt((dist_matrix[0]**2 + dist_matrix[1]**2))
        sort_order = dist_matrix.ravel().argsort()

        xOffset = int((x - xOrigin) / pixelWidth) - size
        yOffset = int((y - yOrigin) / pixelHeight) - size

        data = band.ReadAsArray(xOffset, yOffset, 2*size+1, 2*size+1)
        data_sort = data.ravel()[sort_order]

        density = data_sort[data_sort>0][:9].mean()
    except:
        density = np.NaN

    return density


lat_long_data['density_per_km'] = lat_long_data.apply(lambda x: get_population_density(x.latitude, x.longitude), axis=1)

#lat_long_data[['location','density_per_km']].to_csv('../csv/06_population_density.csv')
csv_it(lat_long_data[['location','density_per_km']], '06_population_density')

with open('./pkl/06_population_density.pkl', 'wb') as fh:
    pickle.dump(lat_long_data[['location','density_per_km']], fh)

#sql_it(lat_long_data[['location','density_per_km']], '06_population_density')

In [10]:
lat_long_data

,location,location_type,country,province,county,city,latitude,longitude,density_per_km
0,Argentina-Buenos_Aires,province,Argentina,Buenos Aires,NaN,NaN,-34.603684,-58.381559,12625.800781
1,Argentina-CABA,province,Argentina,Ciudad de Buenos Aires,NaN,NaN,-34.603684,-58.381559,12625.800781
2,Argentina-Cordoba,province,Argentina,Cordoba,NaN,NaN,-31.420083,-64.188776,2404.108887
3,Argentina-Entre_Rios,province,Argentina,Entre Rios,NaN,NaN,-31.774665,-60.495646,72.495293
4,Argentina-Santa_Fe,province,Argentina,Santa Fe,NaN,NaN,-31.610658,-60.697294,208.092285
5,Argentina-Chaco,province,Argentina,Chaco,NaN,NaN,-27.425717,-59.024378,121.331650
6,Argentina-Corrientes,province,Argentina,Corrientes,NaN,NaN,-27.469213,-58.830635,837.728516
7,Argentina-Formosa,province,Argentina,Formosa,NaN,NaN,-26.185777,-58.175567,41.440811
8,Argentina-Misiones,province,Argentina,Misiones,NaN,NaN,-27.426925,-55.946708,414.057007
9,Argentina-Catamarca,province,Argentina,Catamarca,NaN,NaN,-28.469581,-65.779544,460.153595


In [1]:
import pandas as pd
import numpy as np
import dill
from datetime import timedelta

In [33]:
import matplotlib.pyplot as plt
import seaborn as sns
with open('./pkl/00_cleaned_city_names.pkl', 'rb') as fh:
    location_key = dill.load(fh)
location_key.head(1)

,location,location_type,country,province,county,city
0,Argentina-Buenos_Aires,province,Argentina,Buenos Aires,None,None


In [34]:
with open('./pkl/01_latitude_longitude_google.pkl', 'rb') as fh:
    lat_long = dill.load(fh)
lat_long.head(1)
lat_long = lat_long[['location','latitude','longitude']]
location_key.shape, lat_long.shape
location = pd.merge(location_key, lat_long, on='location', how='inner')
location.head(1)

,location,location_type,country,province,county,city,latitude,longitude
0,Argentina-Buenos_Aires,province,Argentina,Buenos Aires,None,None,-34.603684,-58.381559


In [35]:
location.shape

(1724, 8)

In [36]:
save_it(location[['location','country']], '11_feature_engineering_location')

In [37]:
with open('./pkl/02_airport_information_fallingrain.pkl', 'rb') as fh:
    airport = dill.load(fh)
airport.head(1)

,Airlines,city,FAA,IATA,ICAO,kind,latitude,longitude,max_runway,name,country,state
70,4,BAHIA BLANCA,NaN,BHI,SAZB,Medium,-38.725,-62.169,8579.0,COMANDANTE ESPORA,Argentina,NaN


In [38]:
airport = airport[['city', 'FAA', 'IATA', 'ICAO', 'kind', 'latitude',
       'longitude', 'max_runway', 'name', 'country', 'state']]
airport.head(1)

,city,FAA,IATA,ICAO,kind,latitude,longitude,max_runway,name,country,state
70,BAHIA BLANCA,NaN,BHI,SAZB,Medium,-38.725,-62.169,8579.0,COMANDANTE ESPORA,Argentina,NaN


In [39]:
with open('./pkl/04_merged_latitude_longitude_airport_checkpoint.pkl', 'rb') as fh:
    airport2 = dill.load(fh)
airport2.head(1)
airport2 = airport2[['location', 'latitude', 'longitude', 'airport_index', 'country', 'name', 'FAA', 'IATA',
       'ICAO']]
airport.shape, airport2.shape

((2247, 11), (1714, 9))

In [41]:
airport.kind.unique()
save_it(airport, '11_feature_engineering_airport')

In [43]:
# Closest medium or large airport
airport_coords = airport[['latitude', 'longitude']].values[np.newaxis, :]
places_coords = np.rollaxis(lat_long[['latitude','longitude']].values[np.newaxis, :], 0, -1)
dist_coords = ((places_coords - airport_coords)**2).sum(axis=-1)
min_dist = dist_coords.min(axis=1)

airport_distance = lat_long[['location']].copy()
airport_distance['airport_dist_any'] = min_dist

# Closest large airport
airport_coords = airport.loc[airport.kind=='Large', 
                             ['latitude', 'longitude']].values[np.newaxis, :]
places_coords = np.rollaxis(lat_long[['latitude','longitude']].values[np.newaxis, :], 0, -1)
dist_coords = ((places_coords - airport_coords)**2).sum(axis=-1)
min_dist = dist_coords.min(axis=1)

airport_distance['airport_dist_large'] = min_dist
airport_distance.head()

,location,airport_dist_any,airport_dist_large
0,Argentina-Buenos_Aires,0.003183,0.071514
1,Argentina-CABA,0.003183,0.071514
2,Argentina-Cordoba,0.009602,43.526915
3,Argentina-Entre_Rios,0.000658,13.126461
4,Argentina-Santa_Fe,0.023428,14.983910


In [44]:
save_it(airport_distance, '11_feature_engineering_airport_distance')

In [45]:
with open('./pkl/04_weekly_weather.pkl', 'rb') as fh:
    weather = dill.load(fh)

weather.head(2)

,date,max_temp,max_temp1,max_temp2,location,mean_temp,mean_temp1,mean_temp2,min_temp,min_temp1,min_temp2,dew_point,dew_point1,dew_point2,precipitation,precipitation1,precipitation2,wind,wind1,wind2
0,2015-11-28,94.0,94.0,94.0,Mexico-Guerrero,84.0,84.0,84.0,73.0,75.0,74.0,75.0,78.0,77.0,1.22,0.00,0.0,4.0,4.0,4.0
1,2015-12-05,93.0,94.0,94.0,Mexico-Guerrero,82.0,84.0,84.0,72.0,73.0,75.0,74.0,75.0,78.0,0.00,1.22,0.0,4.0,4.0,4.0


In [46]:
save_it(weather, '11_feature_engineering_weather')

In [47]:
with open('./pkl/05_mosquito_sightings.pkl', 'rb') as fh:
    mosquito = dill.load(fh)
mosquito.head(1)

,vector,occurrence_id,source_type,location_type,polygon_admin,latitude,longitude,year,country,country_id,gaul_ad0,status
34478,Aedes albopictus,34479,unpublished,point,-999,22.89,120.44,2006,Taiwan,TWN,886,NaN


In [49]:
# nearest mosquito sighting
mosquito_coords = mosquito[['latitude', 'longitude']].values[np.newaxis, :]
places_coords = np.rollaxis(lat_long[['latitude','longitude']].values[np.newaxis, :], 0, -1)
dist_coords = ((places_coords - mosquito_coords)**2).sum(axis=-1)
min_dist = dist_coords.min(axis=1)

mosquito_distance = lat_long[['location']].copy()
mosquito_distance['mosquito_dist'] = min_dist

mosquito_distance.head()


,location,mosquito_dist
0,Argentina-Buenos_Aires,0.008009
1,Argentina-CABA,0.008009
2,Argentina-Cordoba,0.000480
3,Argentina-Entre_Rios,1.244226
4,Argentina-Santa_Fe,0.742703


In [50]:
save_it(mosquito_distance, '11_feature_engineering_mosquito_distance')

In [51]:
with open('./pkl/06_population_density.pkl', 'rb') as fh:
    population = dill.load(fh)
population.head(1)


,location,density_per_km
0,Argentina-Buenos_Aires,12625.800781


In [52]:
save_it(population, '11_feature_engineering_population')

In [53]:
print (weather.shape)

model = pd.merge(weather,
                 population,
                 on='location', 
                 how='left')

print (model.shape)

model = pd.merge(model,
                 airport_distance,
                 on='location',
                 how='left')

print (model.shape, model.isnull().sum().max())

model = pd.merge(model,
                 mosquito_distance,
                 on='location',
                 how='left')

print (model.dropna().shape)

model = pd.merge(model,
         location[['location','country']],
         on='location',
         how='left')

print (model.dropna().shape)

(112644, 20)
(113139, 21)
(114129, 23) 0
(116109, 24)
(127989, 25)


## Scrape GDP and PPP

In [54]:
#GDP data
import pandas as pd
import numpy as np
import requests

import re
import time
from io import StringIO
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import os
os.chdir(os.path.expanduser("/home/abhijit/Documents/Abhijit_epidemicModel//notebooks/"))

from csv_pkl_sql import save_it
os.chdir('..')

In [55]:
driver = webdriver.Chrome()

In [56]:
driver.get('https://knoema.com/nwnfkne/world-gdp-ranking-2017-gdp-by-country-data-and-charts')
html = driver.page_source

In [57]:
tables = pd.read_html(html)

In [58]:
tables

[   1980  1990  2000  2005  2013  2014  2015  2016  2017
 0  1980  1990  2000  2005  2013  2014  2015  2016  2017,
    1980  1990  2000  2005  2013  2014  2015  2016  2017
 0  1980  1990  2000  2005  2013  2014  2015  2016  2017,
                    Unnamed: 0     2015     2016     2017
 0                 Afghanistan     20.1     19.5     20.9
 1                     Albania     11.3     11.9     13.2
 2                     Algeria    165.9    159.0    178.3
 3                      Angola    102.6     95.3    124.2
 4         Antigua and Barbuda      1.4      1.5      1.5
 5                   Argentina    642.5    554.1    637.7
 6                     Armenia     10.6     10.6     11.5
 7                   Australia   1232.9   1264.9   1379.5
 8                     Austria    382.3    391.0    416.8
 9                  Azerbaijan     50.8     37.8     40.7
 10                    Bahrain     31.1     32.2     34.9
 11                 Bangladesh    208.3    235.6    261.4
 12             

In [59]:
gdp = tables[2].rename(columns={'Unnamed: 0':'country','Units':'units'})
gdp.head(1)

,country,2015,2016,2017
0,Afghanistan,20.1,19.5,20.9


In [60]:
gdp.columns

Index(['country', '2015', '2016', '2017'], dtype='object')

In [61]:
gdp_ppp = tables[3].rename(columns={'Unnamed: 0':'country','Units':'units'})
gdp_ppp.head(1)

,country,2015,2016,2017
0,Afghanistan,64.3,66.7,69.6


In [62]:
locations = pd.read_pickle('./pkl/00_cleaned_city_names.pkl')
locations.head(1)

,location,location_type,country,province,county,city
0,Argentina-Buenos_Aires,province,Argentina,Buenos Aires,None,None


In [63]:
mask = gdp.country.isin(locations.country.unique())
gdp = gdp.loc[mask]
gdp.shape

(12, 4)

In [64]:
mask = gdp_ppp.country.isin(locations.country.unique())
gdp_ppp = gdp_ppp.loc[mask]
gdp_ppp.shape

(12, 4)

In [65]:
save_it(gdp, '09_GDP_table')

In [66]:
save_it(gdp_ppp, '09_GDP_PPP_table')

In [67]:
import pandas as pd
import numpy as np
import dill
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [69]:
gdp = pd.read_pickle('./pkl/09_GDP_table.pkl')[['country', '2015']]
gdp_ppp = pd.read_pickle('./pkl/09_GDP_PPP_table.pkl')[['country', '2015']]
gdp.head(1)

,country,2015
5,Argentina,642.5


In [70]:
gdp.rename(columns={'2015':'gdp'}, inplace=True)
gdp_ppp.rename(columns={'2015':'gdp_ppp'}, inplace=True)
gdp = pd.merge(gdp, gdp_ppp, on='country')

In [71]:
save_it(gdp, '11_feature_engineering_gdp')

In [72]:
model = pd.merge(model,
                 gdp,
                 on='country',
                 how='left')

print (model.dropna().shape)

model.drop(['country'], axis=1, inplace=True)

save_it(model, '11_features_engineered')

(127989, 27)


In [73]:
model.head(1).T

,0
date,2015-11-28 00:00:00
max_temp,94
max_temp1,94
max_temp2,94
location,Mexico-Guerrero
mean_temp,84
mean_temp1,84
mean_temp2,84
min_temp,73
min_temp1,75


In [9]:
with open('./pkl/03_infection_data_final.pkl', 'rb') as fh:
    infection = dill.load(fh)
infection.head(1)
infection = (infection
             .groupby(['location','date']).sum()
             .reset_index()
            )

infection.sort_values('zika_cases',ascending=False).head(20)


,location,date,zika_cases
342,Brazil,2016-05-28,161241
341,Brazil,2016-05-21,148905
340,Brazil,2016-05-14,138108
339,Brazil,2016-05-07,138108
338,Brazil,2016-04-30,127822
337,Brazil,2016-04-23,120161
30826,El_Salvador,2016-04-23,91896
336,Brazil,2016-04-02,91387
32512,Sudeste,2016-05-28,65328
32511,Sudeste,2016-05-21,61309


In [10]:
model = pd.merge(weather,
                 infection,
                 on=['date','location'],
                 how='left')

model['zika_cases'] = model.zika_cases.fillna(0)

print (model.shape, model.isnull().sum().max())

model = pd.merge(model,
                 population,
                 on='location', 
                 how='left')

print (model.shape, model.isnull().sum().max())

model = pd.merge(model,
                 airport_distance,
                 on='location',
                 how='left')

print (model.shape, model.isnull().sum().max())

model = pd.merge(model,
                 mosquito_distance,
                 on='location',
                 how='left')

print (model.dropna().shape, model.isnull().sum().max())

(112644, 21) 0
(113139, 22) 0
(114129, 24) 0
(116109, 25) 0


In [30]:
#save_it(model, '07_feature_engineering_and_cleaning')
with open('./pkl/07_feature_engineering_and_cleaning.pkl', 'rb') as fh:
    model = dill.load(fh)